In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [3]:
adm = pd.read_csv('Data Files/NSS_Hospital_Inpatient_Admissions.csv', parse_dates=['Encounter Start Date','Encounter End Date'])
mem = pd.read_csv('Data Files/NSS_GreatCare_Members.csv')
pcp_visits = pd.read_csv('Data Files/NSS_PrimaryCare_Office_Visits.csv', parse_dates=['Encounter Date'])

F:\Users\julie\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
adm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   Encounter ID                                         6213 non-null   object        
 1   Patient ID                                           6213 non-null   object        
 2   Encounter Primary Payor ID                           6213 non-null   object        
 3   Encounter Primary Plan ID                            6213 non-null   object        
 4   Facility Provider ID                                 6213 non-null   object        
 5   Attending Provider ID                                5884 non-null   object        
 6   Attending Provider NPI                               5884 non-null   float64       
 7   Facility Provider NPI                                6213 non-null   object        
 8 

In [7]:
adm_df = adm[['Patient ID','Encounter Start Date','Encounter End Date']]
adm_df

,Patient ID,Encounter Start Date,Encounter End Date
0,DS1|10033,2013-01-12,2013-01-14
1,DS1|10043,2014-01-05,2014-01-12
2,DS1|10073,2012-06-13,2012-06-21
3,DS1|10101,2012-12-06,2012-12-09
4,DS1|10118,2015-03-11,2015-03-14
...,...,...,...
6208,DS1|9979,2015-02-06,2015-02-11
6209,DS1|9981,2014-08-07,2014-08-10
6210,DS1|9994,2015-11-21,2015-11-23
6211,DS1|9994,2015-11-24,2015-11-27


In [8]:
groupped = adm_df.groupby(['Patient ID']).size().reset_index().rename(columns={0: 'count'})
uniques = groupped[groupped['count'] == 1]
uniques

,Patient ID,count
0,DS1|10,1
1,DS1|10033,1
2,DS1|10043,1
3,DS1|10073,1
4,DS1|10101,1
...,...,...
4695,DS1|9965,1
4696,DS1|9978,1
4697,DS1|9979,1
4698,DS1|9981,1


In [9]:
adm_df= adm_df[adm_df.duplicated(subset=['Patient ID'], keep=False)]
adm_df


,Patient ID,Encounter Start Date,Encounter End Date
7,DS1|10132,2013-08-14,2013-08-16
8,DS1|10132,2013-03-30,2013-04-01
17,DS1|10253,2011-10-15,2011-11-13
18,DS1|10253,2010-06-02,2010-06-06
19,DS1|10253,2010-04-02,2010-04-16
...,...,...,...
6201,DS1|9943,2013-10-01,2013-10-16
6202,DS1|9943,2013-10-25,2013-10-28
6203,DS1|9943,2014-10-28,2014-10-30
6210,DS1|9994,2015-11-21,2015-11-23


In [10]:
adm_df.sort_values(['Patient ID','Encounter Start Date'])

,Patient ID,Encounter Start Date,Encounter End Date
8,DS1|10132,2013-03-30,2013-04-01
7,DS1|10132,2013-08-14,2013-08-16
19,DS1|10253,2010-04-02,2010-04-16
18,DS1|10253,2010-06-02,2010-06-06
17,DS1|10253,2011-10-15,2011-11-13
...,...,...,...
6201,DS1|9943,2013-10-01,2013-10-16
6202,DS1|9943,2013-10-25,2013-10-28
6203,DS1|9943,2014-10-28,2014-10-30
6210,DS1|9994,2015-11-21,2015-11-23


In [11]:
adm_df[['Encounter Start Date','Encounter End Date']] = adm_df[['Encounter Start Date','Encounter End Date']].\
                                                apply(lambda x: pd.to_datetime(x))
adm_df = adm_df.sort_values(['Patient ID','Encounter Start Date']) 

adm_df['Readmit30']=adm_df.groupby('Patient ID').apply(lambda x : (x['Encounter Start Date'].\
shift(-1)-x['Encounter End Date']).dt.days.le(30).astype(int)).reset_index(0).drop('Patient ID',1)
adm_df.head(50)


F:\Users\julie\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Patient ID,Encounter Start Date,Encounter End Date,Readmit30
8,DS1|10132,2013-03-30,2013-04-01,0
7,DS1|10132,2013-08-14,2013-08-16,0
19,DS1|10253,2010-04-02,2010-04-16,0
18,DS1|10253,2010-06-02,2010-06-06,0
17,DS1|10253,2011-10-15,2011-11-13,0
20,DS1|10291,2010-05-14,2010-05-15,0
21,DS1|10291,2011-02-24,2011-03-05,0
35,DS1|10451,2013-03-20,2013-03-23,1
45,DS1|10451,2013-03-27,2013-04-16,1
37,DS1|10451,2013-04-16,2013-04-26,1


In [12]:
adm_df['Readmit30'].value_counts()

0    1627
1     755
Name: Readmit30, dtype: int64

In [13]:
adm_df['30_day_window'] = adm_df.apply(lambda x:x['Encounter End Date']+pd.offsets.DateOffset(days=30),1)
adm_df

,Patient ID,Encounter Start Date,Encounter End Date,Readmit30,30_day_window
8,DS1|10132,2013-03-30,2013-04-01,0,2013-05-01
7,DS1|10132,2013-08-14,2013-08-16,0,2013-09-15
19,DS1|10253,2010-04-02,2010-04-16,0,2010-05-16
18,DS1|10253,2010-06-02,2010-06-06,0,2010-07-06
17,DS1|10253,2011-10-15,2011-11-13,0,2011-12-13
...,...,...,...,...,...
6201,DS1|9943,2013-10-01,2013-10-16,1,2013-11-15
6202,DS1|9943,2013-10-25,2013-10-28,0,2013-11-27
6203,DS1|9943,2014-10-28,2014-10-30,0,2014-11-29
6210,DS1|9994,2015-11-21,2015-11-23,1,2015-12-23


In [14]:
readmitindex = pd.DataFrame()
indextemp = pd.DataFrame()
readmit = 0
#pull patient numbers that have more then one admission to loop through
for patnum in adm_df['Patient ID'].drop_duplicates():
    window = pd.to_datetime(0) #set window as a datetime to make first loop work
    #create dataframe for each patient number keeping only the final discharge per Start Date
    patient = adm_df[adm_df['Patient ID']==patnum]
    patient = patient.sort_values(['Encounter Start Date','Encounter End Date']).drop_duplicates(subset=['Patient ID','Encounter End Date'], keep='first')
    patient = patient.sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
    
    for rnum in range(0,len(patient)):
        if window < patient.iloc[rnum,1] :
            #test all admissions after rnum
            trigger = patient.iloc[rnum+1:,1]<=(patient.iloc[rnum,2] + pd.DateOffset(days=30)) #column 8=start, 9=end
            #add one if there are Trues
            readmit = readmit + int(trigger.sum() > 0)
            #save 30 day window date to remove dupicate dates
            window = (patient.iloc[rnum,2] + pd.DateOffset(days=30))
            if trigger.sum() > 0:
                readmitindex = readmitindex.append(patient.iloc[rnum])
            else:
                indextemp = indextemp.append(patient.iloc[rnum])

readmitindex['readmit'] = True
indextemp['readmit'] = False
                
index =  readmitindex.append(indextemp)



In [15]:
readmit

527

In [16]:
adm_df2 = adm
adm_df2

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,...,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,...,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0


In [17]:
admissions_dedup = adm.sort_values(by=['Patient ID','Encounter Start Date', 'Encounter End Date']).drop_duplicates(subset=['Patient ID', 'Encounter Start Date'], keep = 'last')
admissions_dedup = adm.sort_values(by=['Patient ID','Encounter End Date', 'Encounter Start Date']).drop_duplicates(subset=['Patient ID', 'Encounter End Date'], keep = 'first')
adm.shape, admissions_dedup.shape

((6213, 23), (6179, 23))

In [18]:
total_hospital_visits = admissions_dedup.shape[0]
total_pcp_visits_in_7_days = 0

for ind, row in admissions_dedup.iterrows():
    patient = row['Patient ID']
    discharge_date = row['Encounter End Date']
    
    for i,r in pcp_visits[pcp_visits['Patient ID']==patient].iterrows():
        if r['Encounter Date'] <= discharge_date + dt.timedelta(days=7) and r['Encounter Date'] >= discharge_date:
            total_pcp_visits_in_7_days += 1
            break
            
print('Total Hospital Visits: ', total_hospital_visits)
print('Total PCP Visits withinin 7 Days of Discharge: ', total_pcp_visits_in_7_days)
print(str.format("PCP Visit Rate: {}%",round(total_pcp_visits_in_7_days/total_hospital_visits*100,2)))

Total Hospital Visits:  6179
Total PCP Visits withinin 7 Days of Discharge:  1863
PCP Visit Rate: 30.15%


In [5]:
npi = mem = pd.read_csv('Data Files/npi.csv')
npi

,Unnamed: 0,npi,enumeration,entry
0,0,1417170523,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 141717..."
1,1,1669599197,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 166959..."
2,2,1407880511,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 140788..."
3,3,1407008907,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 140700..."
4,4,1518045608,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 151804..."
...,...,...,...,...
5410,5411,1336222397,NPI-2,"{'enumeration_type': 'NPI-2', 'number': 133622..."
5411,5412,1336150572,NPI-1,"{'enumeration_type': 'NPI-1', 'number': 133615..."
5412,5413,1417017435,NPI-2,"{'enumeration_type': 'NPI-2', 'number': 141701..."
5413,5414,1790964864,NPI-2,"{'enumeration_type': 'NPI-2', 'number': 179096..."
